# LangChain v1: ReAct Agent with Multiple Tools (QA, Summarization & Web Search)

In [4]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# =====================================================
# 🔐 Load API keys
# =====================================================
load_dotenv(".env")

# =====================================================
# 🔸 Initialize LLM
# =====================================================
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# =====================================================
# ✅ GLOBAL INSTRUCTION (used everywhere)
# =====================================================
SHORT_INSTRUCTION = "Give a SHORT and CLEAR answer. Avoid long explanations."

# =====================================================
# ✅ Tool 1: Simple QA Tool
# =====================================================
qa_prompt = PromptTemplate.from_template(
    "Question: {question}\nInstruction: {instruction}"
)
qa_chain = qa_prompt | llm

@tool
def simple_qa(question: str) -> str:
    """Answer factual questions briefly."""
    response = qa_chain.invoke({
        "question": question,
        "instruction": SHORT_INSTRUCTION
    })
    return response.content.strip()

# =====================================================
# ✅ Tool 2: Summarizer Tool
# =====================================================
summary_prompt = PromptTemplate.from_template(
    "Text:\n{text}\n\nInstruction: {instruction}"
)
summary_chain = summary_prompt | llm

@tool
def summarizer(text: str) -> str:
    """Summarize text briefly."""
    response = summary_chain.invoke({
        "text": text,
        "instruction": SHORT_INSTRUCTION
    })
    return response.content.strip()

# =====================================================
# ✅ Tool 3: Web Search Tool (Tavily)
# =====================================================
tavily_search = TavilySearchResults(max_results=3)

@tool
def web_search(query: str) -> str:
    """Search the internet and return a brief answer."""
    results = tavily_search.invoke({"query": query})

    # Keep search output short
    summaries = []
    for r in results:
        summaries.append(f"- {r.get('title', '')}")

    return "\n".join(summaries[:3])

# =====================================================
# 🧠 Create Agent
# =====================================================
tools = [simple_qa, summarizer, web_search]

agent = create_react_agent(
    model=llm,
    tools=tools
)

# =====================================================
# 🚀 Run user queries
# =====================================================
queries = [
    "What is LangGraph in LangChain?",
    "Summarize this: LangChain helps build LLM apps with tools and agents.",
    "Latest update about Langchain"
]

for query in queries:
    print("\n🧑‍💻 User Query:", query)
    response = agent.invoke({"messages": [("user", query)]})
    print("\n🤖 Agent Response:", response["messages"][-1].content)



C:\Users\15016\AppData\Local\Temp\ipykernel_35184\1423584563.py:84: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(



🧑‍💻 User Query: What is LangGraph in LangChain?

🤖 Agent Response: LangGraph in LangChain is a framework that enables users to create and manage complex workflows by connecting various language model components and tasks in a graph-like structure.

🧑‍💻 User Query: Summarize this: LangChain helps build LLM apps with tools and agents.

🤖 Agent Response: LangChain is a framework for building applications using large language models (LLMs) with tools and agents.

🧑‍💻 User Query: Latest update about Langchain

🤖 Agent Response: LangChain has recently achieved a valuation of $1.25 billion, officially making it a unicorn. This reflects the increasing interest and investment in AI orchestration technologies.


In [3]:
for query in queries:
    print("\n🧑‍💻 User Query:", query)

    response = agent.invoke({"messages": [("user", query)]})

    # Final answer
    print("\n🤖 Final Answer:", response["messages"][-1].content)

    # Tool usage
    used_tools = [
        msg.name for msg in response["messages"]
        if msg.type == "tool"
    ]
    print("🛠 Tools used:", used_tools)



🧑‍💻 User Query: What is LangGraph in LangChain?

🤖 Final Answer: LangGraph in LangChain is a framework that enables users to create and manage complex workflows by connecting various language model components and data sources in a graph-like structure.
🛠 Tools used: ['simple_qa']

🧑‍💻 User Query: Summarize this: LangChain helps build LLM apps with tools and agents.

🤖 Final Answer: LangChain is a framework for building applications using large language models (LLMs) with tools and agents.
🛠 Tools used: ['summarizer']

🧑‍💻 User Query: Latest update about GPT-4o

🤖 Final Answer: The latest update about GPT-4o indicates that its final day is set for February 16th, 2026. Additionally, there has been a deprecation of the chat-gpt-4o-latest version, and discussions are ongoing about the potential retirement of GPT-4 in 2026, along with its implications.
🛠 Tools used: ['web_search']


In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import TavilySearchResults

# 🔐 Load API keys
load_dotenv(".env")
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# 🔸 Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ✅ Tool 1: Simple QA Tool
qa_prompt = PromptTemplate.from_template("Answer clearly: {question}")
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
qa_tool = Tool(
    name="Simple QA",
    func=qa_chain.run,
    description="Answers factual questions clearly"
)

# ✅ Tool 2: Summarizer Tool
summary_prompt = PromptTemplate.from_template("Summarize this text:\n\n{text}")
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
summary_tool = Tool(
    name="Summarizer",
    func=summary_chain.run,
    description="Summarizes long paragraphs or text content"
)

# ✅ Tool 3: Web Search Tool (Tavily)
search_tool = Tool(
    name="Web Search",
    func=TavilySearchResults(max_results=3).run,
    description="Search the internet for current and live information"
)

# 🔧 Wrap all tools in an agent
tools = [qa_tool, summary_tool, search_tool]

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # shows reasoning process
)

# 🚀 Run user queries
queries = [
    "What is LangGraph in LangChain?"
    #"Summarize this: LangChain is a framework to build LLM apps using prompts, memory, tools, and agents.",
    #"Latest news about OpenAI GPT-4o"
]

for query in queries:
    print("\n🧑‍💻 User Query:", query)
    response = agent_executor.run(query)
    print("\n🤖 Agent Response:", response)



🧑‍💻 User Query: What is LangGraph in LangChain?


> Entering new AgentExecutor chain...
I should use the Simple QA tool to answer this factual question.
Action: Simple QA
Action Input: "What is LangGraph in LangChain?"
Observation: LangGraph in LangChain is a graph database that stores and manages language data, including words, phrases, and their relationships. It is a key component of the LangChain platform, allowing for efficient storage, retrieval, and analysis of language-related information.
I now know the final answer
Final Answer: LangGraph in LangChain is a graph database that stores and manages language data.

> Finished chain.

🤖 Agent Response: LangGraph in LangChain is a graph database that stores and manages language data.


In [3]:
response

'OpenAI GPT-4o is a multilingual, multimodal generative pre-trained transformer developed by OpenAI and released in May 2024. It can process and generate text, images, and audio. In August 2024, OpenAI introduced a new feature allowing corporate customers to customize GPT-4o using proprietary company data through fine-tuning.'